In [24]:
import pandas as pd

# read the data
X_train = pd.read_csv('X23241.csv').values[:,1:]
y_train = pd.read_csv('Y23241.csv').values[:,1].ravel().astype(int)

In [25]:
from sklearn.model_selection import train_test_split

# split the data
X_train_sub, X_valid, y_train_sub, y_valid = \
    train_test_split(X_train, y_train, test_size=0.2, random_state=1, stratify=y_train)

print('Train/Valid sizes:', y_train_sub.shape[0], y_valid.shape[0])

Train/Valid sizes: 18592 4649


In [42]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn import metrics

scaler = preprocessing.StandardScaler().fit(X_train_sub[:,0:10])

X_scaled_sub = scaler.transform(X_train_sub[:,0:10])
X_scaled_test = scaler.transform(X_valid[:,0:10])

pca = PCA(n_components=5)
pca.fit(X_scaled_sub)
X_new = pca.transform(X_scaled_sub)
X_new_test = pca.transform(X_scaled_test)

print(pca.explained_variance_ratio_)

forest.fit(X_new, y_train_sub)
print(forest.score(X_new_test, y_valid))


#modeling and grid search
forest = RandomForestClassifier(n_estimators=100,
                                criterion="entropy",
                                random_state=123)

scaled = make_pipeline(StandardScaler(), PCA(n_components=5), forest)

scaled.fit(X_train_sub, y_train_sub)
# pred_test_std = scaled.predict(X_valid)

pca_std = scaled.named_steps["pca"]

#pca_std.explained_variance_ratio_

print(f"Validation Accuracy standardized dataset: {scaled.score(X_valid, y_valid)*100:0.3f}%")

[0.25947817 0.21464207 0.17413273 0.10830627 0.07621763]
0.6857388685738869
Validation Accuracy standardized dataset: 75.027%


In [12]:
print('Best Params: %s' % gs.best_params_) #best parameters

Best Params: {'criterion': 'entropy', 'max_depth': None}


In [13]:
print(f"Training Accuracy: {gs.best_estimator_.score(X_train_sub, y_train_sub)*100:0.3f}%")
print(f"Validation Accuracy: {gs.best_estimator_.score(X_valid, y_valid)*100:0.3f}%")

Training Accuracy: 100.000%
Validation Accuracy: 81.265%


In [14]:
forest_best = RandomForestClassifier(n_estimators=100,
                                     criterion="entropy",
                                     random_state=123)

%timeit forest_best.fit(X_train_sub, y_train_sub) #training(fitting) time
%timeit forest_best.score(X_valid, y_valid) #test time

6.21 s ± 84.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
89.7 ms ± 264 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
